In [19]:
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
import openpyxl
from sklearn_extra.cluster import KMedoids
from sklearn.decomposition import PCA
from scipy.stats import chi2_contingency

# Load the dataset from an Excel file
df = pd.read_excel('Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx', sheet_name='Genus-level microbiota')
print(df.columns.size)

# Assuming the dataset has a column 'Sample_ID' that contains the identifiers
# If 'Sample_ID' is not the first column, adjust the column name accordingly
df['Fertility_Status'] = df['Sample ID'].apply(lambda x: 'fertile' if x.startswith('CON') else 'infertile')

# Assuming all other columns are bacterial phyla
feature_columns = df.columns.drop(['Sample ID', 'Fertility_Status'])  # Exclude non-feature columns

# Extract features (X)
X = df[feature_columns]

from scipy.stats import ttest_ind
import numpy as np

significant_phyla = []
p_threshold = 0.05  # Adjust the p-value threshold as needed

# Save one sample from each fertility status
sample_fertile = df[df['Fertility_Status'] == 'fertile'].sample(1)
sample_infertile = df[df['Fertility_Status'] == 'infertile'].sample(1)

for phylum in feature_columns:
    t_stat, p_val = ttest_ind(df[df['Fertility_Status'] == 'fertile'][phylum],
                              df[df['Fertility_Status'] == 'infertile'][phylum],
                              equal_var=False, nan_policy='omit')
    if p_val < p_threshold:
        significant_phyla.append(phylum)

# Keep only significant phyla
X_refined = df[significant_phyla]

# print refined dataset
print(X_refined.columns.size)

# Existing code for normalization and setup
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_refined)

hclust = AgglomerativeClustering(n_clusters=10)
clusters = hclust.fit_predict(X_scaled)

from sklearn.decomposition import PCA

# Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_pca = pca.fit_transform(X_scaled)

# You can also check how many components are retained
print("Number of PCA components:", pca.n_components_)

# Apply PAM clustering
pam = KMedoids(n_clusters=10, random_state=0)  # Adjust n_clusters based on your needs
clusters = pam.fit_predict(X_pca)

# Add cluster labels to your DataFrame
df['Cluster'] = clusters

# Group by Fertility Status and Cluster
cluster_distribution = df.groupby(['Fertility_Status', 'Cluster']).size().unstack(fill_value=0)
print("Cluster distribution by fertility status:")
print(cluster_distribution)

# Analyzing characteristics of each cluster
for cluster in range(pam.n_clusters):
    print(f"\nCharacteristics of Cluster {cluster}:")
    cluster_data = df[df['Cluster'] == cluster]
    average_phyla_abundance = cluster_data[feature_columns].mean()
    #print only non zero values
    average_phyla_abundance = average_phyla_abundance[average_phyla_abundance > 0]
    print(average_phyla_abundance)

# Cluster distribution by fertility status
cluster_distribution = df.groupby(['Fertility_Status', 'Cluster']).size().unstack(fill_value=0)

print("Cluster distribution by fertility status:")
print(cluster_distribution)

# Apply Chi-squared test
chi2, p, dof, expected = chi2_contingency(cluster_distribution)

print(f"\nChi-squared test results:")
print(f"Chi2 statistic: {chi2}, p-value: {p}")

from scipy.stats import fisher_exact

# Fisher's Exact Test for each cluster
from scipy.stats import fisher_exact

# Fisher's Exact Test for each cluster
for cluster in range(pam.n_clusters):
    # Presence in the cluster
    cluster_present = cluster_distribution.get(cluster, pd.Series([0, 0]))

    # Absence in the cluster
    cluster_absent = cluster_distribution.sum(axis=1) - cluster_present
    cluster_absent.name = 'Not in Cluster'

    # Create a (2, 2) contingency table
    contingency_table = pd.concat([cluster_present, cluster_absent], axis=1)

    # Apply Fisher's Exact Test
    oddsratio, p_value = fisher_exact(contingency_table)
    print(f"Cluster {cluster}: Fisher's Exact Test p-value = {p_value}")



805
38
Number of PCA components: 22
Cluster distribution by fertility status:
Cluster           0  1  2  3  4  5  6  7   8  9
Fertility_Status                               
fertile           2  2  3  1  3  0  0  2   1  0
infertile         8  1  0  6  2  3  4  0  12  6

Characteristics of Cluster 0:
Finegoldia                 9.91538
Peptoniphilus              7.22509
Anaerococcus               8.57815
Campylobacter              3.04248
Streptococcus              5.00924
                            ...   
Adlercreutzia              0.00011
Bernardetia                0.00019
Candidatus.Hepatoplasma    0.00005
Algibacter                 0.00018
Zunongwangia               0.00005
Length: 603, dtype: float64

Characteristics of Cluster 1:
Finegoldia               8.841733
Peptoniphilus            5.114033
Anaerococcus             4.116300
Campylobacter           17.035767
Streptococcus            9.190300
                          ...    
Desulfobacterium         0.000600
Candidatus.Cardin